In [ ]:
# Pandas is for formatting the data into matrices, making it easier to query
# Numpy is used for some mathematical functions, such as matrix multiplication
# Datetime is used to assess how long it takes code blocks to run
# Fiona is used for loading country shapefiles, and for exporting the results as a shapefile
# Pickle is for storing and python objects in memory for easy access later
# GeoPandas is for programmatically mapping

import pandas as pd
import numpy as np
import datetime
import fiona
# import shapely, can be used for spatial computations
import pickle
# import scipy as sp, can be used for running statistical tests
import geopandas as gpd

# Matplotlib inline to visualize maps in jupyter notebook
# Use PdfPages to print plots to a pdf
# pyplot helps with laying out plots
%matplotlib inline
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt

# Set options to see more of the data tables at a time

pd.options.display.max_rows = 200
pd.options.display.max_columns = 200

In [ ]:
# Load IPUMS census data for China, includes Industry General response
# i.e. which industry employed people report working for

f = open("/Users/nathansuberi/Desktop/WRI_Programming/Py_Scripts/IPUMS_files/Chinese_INDGEN_ipumsi.dat", "r")
# Another load method:
# ipums = np.fromfile("/Users/nathansuberi/Downloads/ipumsi_00003.dat")

data = f.readlines()
dataex = pd.Series(data)
print("number of records: ", len(data))
print("example of a household record: ", data[1])

# The IPUMS survey data are provided as coded strings of numbers
# The following code blocks create the dictionaries that are used to decode these strings
# And create data frames with human readable information

# All of the conversion files are adapted from the conversion codes
# Provided by IPUMS with their data download
# And stored in the folder below:
file_loc = "/Users/nathansuberi/Desktop/WRI_Programming/Py_Scripts/IPUMS_Conversions/"

In [ ]:
# Create dictionary for country names
country_names = pd.read_csv(file_loc + "country_names.csv", header=None)

ref = country_names.apply(lambda row: row[0].split("   "), axis=1)
code, country = ref.apply(lambda row :row[0]), ref.apply(lambda row :row[1])
country_names = pd.DataFrame({"code":code, "country":country})
country_names.set_index(["code"], inplace=True)
print(country_names.loc["156"])

# Create dictionary for Industry of Employment
# Have to change the sep, because there are commas in these column titles
ind_gen = pd.read_csv(file_loc + "ind_gen.csv", header=None, sep="\t")

ref = ind_gen.apply(lambda row: row[0].split("   "), axis=1)
code, ind_gen = ref.apply(lambda row :row[0]), ref.apply(lambda row :row[1])
ind_gen = pd.DataFrame({"code":code, "ind_gen":ind_gen})
ind_gen.set_index(["code"], inplace=True)
print(ind_gen.loc["010"])

# Create dictionary for Employment Status
emp_status = pd.read_csv(file_loc + "emp_stat.csv", header=None)

ref = emp_status.apply(lambda row: row[0].split("   "), axis=1)
code, emp_stat = ref.apply(lambda row :row[0]), ref.apply(lambda row :row[1])
emp_status = pd.DataFrame({"code":code, "emp_stat":emp_stat})
emp_status.set_index(["code"], inplace=True)
#emp_status

# Create dictionary for Detailed Employment Status
# Have to change the sep, because there are commas in these column titles
emp_status_d = pd.read_csv(file_loc + "emp_stat_detailed.csv", header=None, sep="\t")

ref = emp_status_d.apply(lambda row: row[0].split("   "), axis=1)
code, emp_stat_d = ref.apply(lambda row :row[0]), ref.apply(lambda row :row[1])
emp_status_d = pd.DataFrame({"code":code, "emp_stat_d":emp_stat_d})
emp_status_d.set_index(["code"], inplace=True)
#emp_status_d

# Create dictionary for Class Work
# Have to change the sep, because there are commas in these column titles
class_wk = pd.read_csv(file_loc + "class_wk.csv", header=None, sep="\t")

ref = class_wk.apply(lambda row: row[0].split("   "), axis=1)
code, class_wk = ref.apply(lambda row :row[0]), ref.apply(lambda row :row[1])
class_wk = pd.DataFrame({"code":code, "class_wk":class_wk})
class_wk.set_index(["code"], inplace=True)
# class_wk

# Create dictionary for Detailed Class Work Status
# Have to change the sep, because there are commas in these column titles
class_wk_d = pd.read_csv(file_loc + "class_wk_detailed.csv", header=None, sep="\t")

ref = class_wk_d.apply(lambda row: row[0].split("   "), axis=1)
code, class_wk_d = ref.apply(lambda row :row[0]), ref.apply(lambda row :row[1])
class_wk_d = pd.DataFrame({"code":code, "class_wk_d":class_wk_d})
class_wk_d.set_index(["code"], inplace=True)
#class_wk_d

In [ ]:
# Now create decoded dataframe
# Takes 22 minutes for all >30,000,000 of the Chinese records

start_time = datetime.datetime.now()
print("start creating data frame:", start_time)

datadf = pd.DataFrame({
    "Country": country_names.loc[dataex.apply(lambda row: row[0:3])]["country"].values,
    "Year": dataex.apply(lambda row: str(row[3:7])).values,
    "GeoLev2": dataex.apply(lambda row: str(row[7:16])).values,
    "IndGen": ind_gen.loc[dataex.apply(lambda row: row[16:19])]["ind_gen"].values
}, index=range(0,len(dataex)))

end_time = datetime.datetime.now()
print("end", end_time - start_time)

print(datadf)

# Use Pickle to save the decoded Chinese census responses for re-use
china_indgen_filename  = '/Users/nathansuberi/Desktop/WRI_Programming/Py_Scripts/IPUMS_GeoLev2_Pickles/China_Indgen_DF.pkl'

# Dump to file
pickle.dump(datadf, open(china_indgen_filename, 'wb'))


In [ ]:
## Can start analysis here, once original IPUMS data is decoded and saved to a pickle

In [ ]:
# Load Chinese census response data from pickle file
china_indgen_filename  = '/Users/nathansuberi/Desktop/WRI_Programming/Py_Scripts/IPUMS_GeoLev2_Pickles/China_Indgen_DF.pkl'

start_time = datetime.datetime.now()
print("start loading data frame:", start_time)

china_indgen_df = pickle.load(open(china_indgen_filename, 'rb'))

end_time = datetime.datetime.now()
print("finished in: ", end_time - start_time)

In [ ]:
# Examine the Chinese census response data 
# regarding General Industry people are employed in
china_indgen_df

In [ ]:
# Create a MultiIndex using the GeoLev2 and Year to organize the census responses
# https://pandas.pydata.org/pandas-docs/stable/advanced.html

start_time = datetime.datetime.now()
print("start creating multi-index:", start_time)

admin_year_tuples = list(zip(*[china_indgen_df["GeoLev2"],china_indgen_df["Year"]]))
indgen_multi_index = pd.MultiIndex.from_tuples(admin_year_tuples, names=["GeoLev2", "Year"])

china_indgen_df.index = indgen_multi_index

end_time = datetime.datetime.now()
print("finished in: ", end_time - start_time)

In [ ]:
# Use this MultiIndex on the census response data to loop over that data
# And store the distribution of employment for each GeoLev2 in each year

# Option: calculate employment as counts or proportions per GeoLev2
# Counts are preferable for this application, as I want to downscale the total value
# I think is generated in that GeoLev2 for a certain industry
# I am using national trade accounts, and distributing total production value
# According to number of people employed in relevant industries in each GeoLev2 
# relative to national total employment in relevant industries.

# def return_props(cts,key):
#     return round(cts[key]/cts.sum(), 4) if key in cts.keys() else 0

# The ternary operation here checks if the employment type is represented in the GeoLev2 & Year pair
# If not, it returns 0. This way all jobs are always accounted for.
# Multiply by 100 because we only have 1% of the Chinese census
# Use round function to make sure we return an integer count
def return_counts(cts,key,sample_perc):
    # print("before: ", round(cts[key],0) if key in cts.keys() else 0)
    # print("after: ", round((100/sample_perc)*cts[key],0) if key in cts.keys() else 0)
    return(round((100/sample_perc)*cts[key],0) if key in cts.keys() else 0)

start_time = datetime.datetime.now()
print("start creating rows:", start_time)

# Create an empty list to store employment numbers by GeoLev2 & Year (one record for each pair)

rows = []
# Percentage of census included in China IPUMS microdata sample
china_sample_perc = 1
# Loop over all geolev2s
for gl2 in indgen_multi_index.levels[0]:
    # Loop over all years
    for yr in indgen_multi_index.levels[1]:
        # print(gl2)
        # print(yr)
        
        # This line accounts for fact that not all geolev2's have data for all years
        # Ex: china_indgen_df.loc['156042090'] does not contain data for all years
        # And will throw an error if the program tries to access that index 
        # with the command china_indgen_df.loc[gl2].loc[yr]
        if(yr in china_indgen_df.loc[gl2]["Year"].unique()):
            cts = china_indgen_df.loc[gl2].loc[yr]["IndGen"].value_counts()
            total = cts.sum()

            row = {
                "GeoLev2":gl2,
                "Year":yr,
                "Total Population in Sample": total,
                
                "NIU (not in universe)": return_counts(cts, "\"NIU (not in universe)\"",china_sample_perc), 
                "Agriculture, fishing, and forestry": return_counts(cts, "\"Agriculture, fishing, and forestry\"",china_sample_perc), 
                "Manufacturing": return_counts(cts, "\"Manufacturing\"",china_sample_perc), 
                "Construction": return_counts(cts, "\"Construction\"",china_sample_perc), 
                "Wholesale and retail trade": return_counts(cts, "\"Wholesale and retail trade\"",china_sample_perc), 
                "Transportation, storage and communications": return_counts(cts, "\"Transportation, storage and communications\"",china_sample_perc), 
                "Education": return_counts(cts, "\"Education\"",china_sample_perc), 
                "Mining": return_counts(cts, "\"Mining\"",china_sample_perc), 
                "Public administration and defense": return_counts(cts, "\"Public administration and defense\"",china_sample_perc), 
                "Other services": return_counts(cts, "\"Other services\"",china_sample_perc), 
                "Hotels and restaurants": return_counts(cts, "\"Hotels and restaurants\"",china_sample_perc), 
                "Health and social work": return_counts(cts, "\"Health and social work\"",china_sample_perc), 
                "Unknown": return_counts(cts, "\"Unknown\"",china_sample_perc), 
                "Electricity, gas and water": return_counts(cts, "\"Electricity, gas and water\"",china_sample_perc), 
                "Real estate and business services": return_counts(cts, "\"Real estate and business services\"",china_sample_perc), 
                "Financial services and insurance": return_counts(cts, "\"Financial services and insurance\"",china_sample_perc), 
                "Other industry, n.e.c.": return_counts(cts, "\"Other industry, n.e.c.\"",china_sample_perc),
                "Services, not specified": return_counts(cts, "\"Services, not specified\"",china_sample_perc),
                "Private household services": return_counts(cts, "\"Private household services\"",china_sample_perc),
                "Response suppressed": return_counts(cts, "\"Response suppressed\"",china_sample_perc),
            }
            rows.append(row)

# Create the dataframe of all GeoLev2 & Year combo, employment counts         
employment_by_geolev2 = pd.DataFrame(rows)

# Use Pickle to save the decoded Chinese census responses for re-use
china_employment_by_geolev2_filename  = '/Users/nathansuberi/Desktop/WRI_Programming/Py_Scripts/IPUMS_GeoLev2_Pickles/China_IndGen_by_GeoLev2_DF.pkl'

# Dump to file
pickle.dump(employment_by_geolev2, open(china_employment_by_geolev2_filename, 'wb'))

end_time = datetime.datetime.now()
print("finished in: ", end_time - start_time)

In [ ]:
# Load Chinese employment summary data from pickle file
china_employment_by_geolev2_filename  = '/Users/nathansuberi/Desktop/WRI_Programming/Py_Scripts/IPUMS_GeoLev2_Pickles/China_IndGen_by_GeoLev2_DF.pkl'

start_time = datetime.datetime.now()
print("start loading data frame:", start_time)

china_employment_by_geolev2 = pickle.load(open(china_employment_by_geolev2_filename, 'rb'))
# Create MultiIndex by GeoLev2 and Year for easier querying

admin_year_tuples = list(zip(*[china_employment_by_geolev2["GeoLev2"], china_employment_by_geolev2["Year"]]))

indgen_geolev2_multi_index = pd.MultiIndex.from_tuples(admin_year_tuples, names=["GeoLev2", "Year"])
china_employment_by_geolev2.index = indgen_geolev2_multi_index

# Don't need these columns anymore, as the information is stored in the MultiIndex
china_employment_by_geolev2.drop(["GeoLev2", "Year"], inplace=True, axis=1)

end_time = datetime.datetime.now()
print("finished in: ", end_time - start_time)

In [ ]:
# Inspect employment summary data
china_employment_by_geolev2

In [ ]:
# Can now request data by GeoLev2 code and year
china_employment_by_geolev2.loc["156011001"].loc["2000"]

In [ ]:
# Import WorldGeoLev2 shapefile to attach the results of the jobs / trade analysis
world_geolev2_boundary_shapefile = "/Users/nathansuberi/Desktop/WRI_Programming/Py_Scripts/world_geolev2/world_geolev2.shp"

c = fiona.open(world_geolev2_boundary_shapefile)
# Create an iterable collection that can be looped over
coll = iter(c)

# Only choose those polygons that are in China, and have a value for their GEOLEVEL2
china_geolev2_polys = []

for poly in coll:
    # print(poly["properties"].keys())
    if((poly["properties"]["CNTRY_NAME"] == "China") and (poly["properties"]["GEOLEVEL2"] != None)):
            china_geolev2_polys.append(poly)

In [ ]:
# Should be 199 to match 
print(len(china_geolev2_polys) == len(china_employment_by_geolev2.index.levels[0]))
print(len(china_geolev2_polys))

In [ ]:
# QA/QC: Check to see that the 
valid_shp = []
invalid_shp = []

for poly in china_geolev2_polys:
    if(poly["properties"]["GEOLEVEL2"] not in china_employment_by_geolev2.index.levels[0]):
        print("uh oh...")
        print(poly["properties"]["GEOLEVEL2"])
        invalid_shp.append(poly["properties"]["GEOLEVEL2"])
    else:
        valid_shp.append(poly["properties"]["GEOLEVEL2"])
        

In [ ]:
# One geolev2 doesn't match between the two
len(valid_shp)

In [ ]:
# Check to see which china_employment_by_geolev2is invalid
valid_emp = []
invalid_emp = []
# Loop over the GeoLev2s in the IPUMS census data, store the ones that match between the
# Shapefile and the IPUMS data
# A bit strange, because they both come from IPUMS

### FLAG THIS FOR PAT @ IPUMS

for gl2 in china_employment_by_geolev2.index.levels[0]:
    if(gl2 not in valid_shp):
        print("uh oh...")
        print(gl2)
        invalid_emp.append(gl2)
    else:
        valid_emp.append(gl2)

In [ ]:
# Check to see that the valid geolev2s are the same
sorted(valid_emp) == sorted(valid_shp)

In [ ]:
final_china_geolev2_polys = []
# Loop over the china_geolev2_polys, if they have employment data (i.e. in valid_shp list)
# Attach the employment data to the poly
for poly in china_geolev2_polys:
    if(poly["properties"]["GEOLEVEL2"] in valid_shp):
        # Adds employment data for all years to the shapefile matching each GeoLev2
        poly["properties"]["job_data"] = china_employment_by_geolev2.loc[poly["properties"]["GEOLEVEL2"]]
        final_china_geolev2_polys.append(poly)

In [ ]:
# Inspect the first GeoLev2's properties, corresponds to Beijing Municipality
final_china_geolev2_polys[0]["properties"]

In [ ]:
# Need to adjust the schema to reflect the true format of data to be exported





# Use fiona to export to a shapefile, for viz in Carto
## Data schema:
## **** Creating the schema would need to be automated as well... think on it

#### Add in analysis above, schema and export below, 
# for % change in trade and CO2 emissions

schema = {
    'geometry':'Polygon',
    'properties': {
        'GEOLEVEL2': 'int',
        
        '19_t_ag': 'float',
        '19_c_ag': 'float',

        '19_t_fsh': 'float',
        '19_c_fsh': 'float',

        '19_t_maq': 'float',
        '19_c_maq': 'float',

        '19_t_fab': 'float',
        '19_c_fab': 'float',

        '19_t_tawa': 'float',
        '19_c_tawa': 'float',

        '19_t_wap': 'float',
        '19_c_wap': 'float',

        '19_t_pcnmp': 'float',
        '19_c_pcnmp': 'float',

        '19_t_mp': 'float',
        '19_c_mp': 'float',

        '19_t_eam': 'float',
        '19_c_eam': 'float',

        '19_t_te': 'float',
        '19_c_te': 'float',

        '19_t_om': 'float',
        '19_c_om': 'float',

        '19_t_rcycl': 'float',
        '19_c_rcycl': 'float',

        '19_t_egw': 'float',
        '19_c_egw': 'float',

        '19_t_con': 'float',
        '19_c_con': 'float',

        '19_t_mar': 'float',
        '19_c_mar': 'float',

        '19_t_wt': 'float',
        '19_c_wt': 'float',

        '19_t_rt': 'float',
        '19_c_rt': 'float',

        # Careful, misspelled in EORA, restraurants
        '19_t_har': 'float',
        '19_c_har': 'float',

        '19_t_xprt': 'float',
        '19_c_xprt': 'float',

        '19_t_pat': 'float',
        '19_c_pat': 'float',

        # Careful, misspelled in EORA, finacial
        '19_t_fiba': 'float',
        '19_c_fiba': 'float',

        '19_t_pa': 'float',
        '19_c_pa': 'float',

        '19_t_ehos': 'float',
        '19_c_ehos': 'float',

        '19_t_ph': 'float',
        '19_c_ph': 'float',

        '19_t_oth': 'float',
        '19_c_oth': 'float',
        
        # There are no carbon emissions from re-export & re-import
        '19_t_reri': 'float',
        # '19_c_reri': 'float',



        '20_t_ag': 'float',
        '20_c_ag': 'float',

        '20_t_fsh': 'float',
        '20_c_fsh': 'float',

        '20_t_maq': 'float',
        '20_c_maq': 'float',

        '20_t_fab': 'float',
        '20_c_fab': 'float',

        '20_t_tawa': 'float',
        '20_c_tawa': 'float',

        '20_t_wap': 'float',
        '20_c_wap': 'float',

        '20_t_pcnmp': 'float',
        '20_c_pcnmp': 'float',

        '20_t_mp': 'float',
        '20_c_mp': 'float',

        '20_t_eam': 'float',
        '20_c_eam': 'float',

        '20_t_te': 'float',
        '20_c_te': 'float',

        '20_t_om': 'float',
        '20_c_om': 'float',

        '20_t_rcycl': 'float',
        '20_c_rcycl': 'float',

        '20_t_egw': 'float',
        '20_c_egw': 'float',

        '20_t_con': 'float',
        '20_c_con': 'float',

        '20_t_mar': 'float',
        '20_c_mar': 'float',

        '20_t_wt': 'float',
        '20_c_wt': 'float',

        '20_t_rt': 'float',
        '20_c_rt': 'float',

        # Careful, misspelled in EORA, restraurants
        '20_t_har': 'float',
        '20_c_har': 'float',

        '20_t_xprt': 'float',
        '20_c_xprt': 'float',

        '20_t_pat': 'float',
        '20_c_pat': 'float',

        # Careful, misspelled in EORA, finacial
        '20_t_fiba': 'float',
        '20_c_fiba': 'float',

        '20_t_pa': 'float',
        '20_c_pa': 'float',

        '20_t_ehos': 'float',
        '20_c_ehos': 'float',

        '20_t_ph': 'float',
        '20_c_ph': 'float',

        '20_t_oth': 'float',
        '20_c_oth': 'float',
        
        # There are no carbon emissions from re-export & re-import
        '20_t_reri': 'float',
        # '20_c_reri': 'float',
        
        
        
        'pc_t_ag': 'float',
        'pc_c_ag': 'float',

        'pc_t_fsh': 'float',
        'pc_c_fsh': 'float',

        'pc_t_maq': 'float',
        'pc_c_maq': 'float',

        'pc_t_fab': 'float',
        'pc_c_fab': 'float',

        'pc_t_tawa': 'float',
        'pc_c_tawa': 'float',

        'pc_t_wap': 'float',
        'pc_c_wap': 'float',

        'pc_t_pcnmp': 'float',
        'pc_c_pcnmp': 'float',

        'pc_t_mp': 'float',
        'pc_c_mp': 'float',

        'pc_t_eam': 'float',
        'pc_c_eam': 'float',

        'pc_t_te': 'float',
        'pc_c_te': 'float',

        'pc_t_om': 'float',
        'pc_c_om': 'float',

        'pc_t_rcycl': 'float',
        'pc_c_rcycl': 'float',

        'pc_t_egw': 'float',
        'pc_c_egw': 'float',

        'pc_t_con': 'float',
        'pc_c_con': 'float',

        'pc_t_mar': 'float',
        'pc_c_mar': 'float',

        'pc_t_wt': 'float',
        'pc_c_wt': 'float',

        'pc_t_rt': 'float',
        'pc_c_rt': 'float',

        # Careful, misspelled in EORA, restraurants
        'pc_t_har': 'float',
        'pc_c_har': 'float',

        'pc_t_xprt': 'float',
        'pc_c_xprt': 'float',

        'pc_t_pat': 'float',
        'pc_c_pat': 'float',

        # Careful, misspelled in EORA, finacial
        'pc_t_fiba': 'float',
        'pc_c_fiba': 'float',

        'pc_t_pa': 'float',
        'pc_c_pa': 'float',

        'pc_t_ehos': 'float',
        'pc_c_ehos': 'float',

        'pc_t_ph': 'float',
        'pc_c_ph': 'float',

        'pc_t_oth': 'float',
        'pc_c_oth': 'float',
        
        # There are no carbon emissions from re-export & re-import
        'pc_t_reri': 'float',
        #'pc_c_reri': 'float'
        
    }
}

# Write out to shapefile

file_destination = "/Users/nathansuberi/Desktop/WRI_Programming/Py_Scripts/Country_EORA_IPUMS_Downscale"

with fiona.open(file_destination + "/china_trade_and_CO2_impacts", "w", "ESRI Shapefile", schema) as c:
    for poly in final_china_geolev2_polys:
        
        # Store properties for easier access
        props = poly["properties"]
        
        c.write({
            'geometry':poly["geometry"],
            'properties': {
                'GEOLEVEL2': props["GEOLEVEL2"],
                
                '19_t_ag': props["downscaled_trade_value"]["Agriculture"].loc["1990"],
                '19_c_ag': props["downscaled_impacts_from_production"]["Agriculture"].loc["1990"],

                '19_t_fsh': props["downscaled_trade_value"]["Fishing"].loc["1990"],
                '19_c_fsh': props["downscaled_impacts_from_production"]["Fishing"].loc["1990"],

                '19_t_maq': props["downscaled_trade_value"]["Mining and Quarrying"].loc["1990"],
                '19_c_maq': props["downscaled_impacts_from_production"]["Mining and Quarrying"].loc["1990"],

                '19_t_fab': props["downscaled_trade_value"]["Food & Beverages"].loc["1990"],
                '19_c_fab': props["downscaled_impacts_from_production"]["Food & Beverages"].loc["1990"],

                '19_t_tawa': props["downscaled_trade_value"]["Textiles and Wearing Apparel"].loc["1990"],
                '19_c_tawa': props["downscaled_impacts_from_production"]["Textiles and Wearing Apparel"].loc["1990"],

                '19_t_wap': props["downscaled_trade_value"]["Wood and Paper"].loc["1990"],
                '19_c_wap': props["downscaled_impacts_from_production"]["Wood and Paper"].loc["1990"],

                '19_t_pcnmp': props["downscaled_trade_value"]["Petroleum, Chemical and Non-Metallic Mineral Products"].loc["1990"],
                '19_c_pcnmp': props["downscaled_impacts_from_production"]["Petroleum, Chemical and Non-Metallic Mineral Products"].loc["1990"],

                '19_t_mp': props["downscaled_trade_value"]["Metal Products"].loc["1990"],
                '19_c_mp': props["downscaled_impacts_from_production"]["Metal Products"].loc["1990"],

                '19_t_eam': props["downscaled_trade_value"]["Electrical and Machinery"].loc["1990"],
                '19_c_eam': props["downscaled_impacts_from_production"]["Electrical and Machinery"].loc["1990"],

                '19_t_te': props["downscaled_trade_value"]["Transport Equipment"].loc["1990"],
                '19_c_te': props["downscaled_impacts_from_production"]["Transport Equipment"].loc["1990"],

                '19_t_om': props["downscaled_trade_value"]["Other Manufacturing"].loc["1990"],
                '19_c_om': props["downscaled_impacts_from_production"]["Other Manufacturing"].loc["1990"],

                '19_t_rcycl': props["downscaled_trade_value"]["Recycling"].loc["1990"],
                '19_c_rcycl': props["downscaled_impacts_from_production"]["Recycling"].loc["1990"],

                '19_t_egw': props["downscaled_trade_value"]["Electricity, Gas and Water"].loc["1990"],
                '19_c_egw': props["downscaled_impacts_from_production"]["Electricity, Gas and Water"].loc["1990"],

                '19_t_con': props["downscaled_trade_value"]["Construction"].loc["1990"],
                '19_c_con': props["downscaled_impacts_from_production"]["Construction"].loc["1990"],

                '19_t_mar': props["downscaled_trade_value"]["Maintenance and Repair"].loc["1990"],
                '19_c_mar': props["downscaled_impacts_from_production"]["Maintenance and Repair"].loc["1990"],

                '19_t_wt': props["downscaled_trade_value"]["Wholesale Trade"].loc["1990"],
                '19_c_wt': props["downscaled_impacts_from_production"]["Wholesale Trade"].loc["1990"],

                '19_t_rt': props["downscaled_trade_value"]["Retail Trade"].loc["1990"],
                '19_c_rt': props["downscaled_impacts_from_production"]["Retail Trade"].loc["1990"],

                # Careful, misspelled in EORA, restraurants
                '19_t_har': props["downscaled_trade_value"]["Hotels and Restraurants"].loc["1990"],
                '19_c_har': props["downscaled_impacts_from_production"]["Hotels and Restraurants"].loc["1990"],

                '19_t_xprt': props["downscaled_trade_value"]["Transport"].loc["1990"],
                '19_c_xprt': props["downscaled_impacts_from_production"]["Transport"].loc["1990"],

                '19_t_pat': props["downscaled_trade_value"]["Post and Telecommunications"].loc["1990"],
                '19_c_pat': props["downscaled_impacts_from_production"]["Post and Telecommunications"].loc["1990"],

                # Careful, misspelled in EORA, finacial
                '19_t_fiba': props["downscaled_trade_value"]["Finacial Intermediation and Business Activities"].loc["1990"],
                '19_c_fiba': props["downscaled_impacts_from_production"]["Finacial Intermediation and Business Activities"].loc["1990"],

                '19_t_pa': props["downscaled_trade_value"]["Public Administration"].loc["1990"],
                '19_c_pa': props["downscaled_impacts_from_production"]["Public Administration"].loc["1990"],

                '19_t_ehos': props["downscaled_trade_value"]["Education, Health and Other Services"].loc["1990"],
                '19_c_ehos': props["downscaled_impacts_from_production"]["Education, Health and Other Services"].loc["1990"],

                '19_t_ph': props["downscaled_trade_value"]["Private Households"].loc["1990"],
                '19_c_ph': props["downscaled_impacts_from_production"]["Private Households"].loc["1990"],

                '19_t_oth': props["downscaled_trade_value"]["Others"].loc["1990"],
                '19_c_oth': props["downscaled_impacts_from_production"]["Others"].loc["1990"],
                
                # There are no carbon emissions from re-export & re-import
                '19_t_reri': props["downscaled_trade_value"]["Re-export & Re-import"].loc["1990"],
                #'19_c_reri': props["downscaled_impacts_from_production"]["Re-export & Re-import"].loc["1990"],




                '20_t_ag': props["downscaled_trade_value"]["Agriculture"].loc["2000"],
                '20_c_ag': props["downscaled_impacts_from_production"]["Agriculture"].loc["2000"],

                '20_t_fsh': props["downscaled_trade_value"]["Fishing"].loc["2000"],
                '20_c_fsh': props["downscaled_impacts_from_production"]["Fishing"].loc["2000"],

                '20_t_maq': props["downscaled_trade_value"]["Mining and Quarrying"].loc["2000"],
                '20_c_maq': props["downscaled_impacts_from_production"]["Mining and Quarrying"].loc["2000"],

                '20_t_fab': props["downscaled_trade_value"]["Food & Beverages"].loc["2000"],
                '20_c_fab': props["downscaled_impacts_from_production"]["Food & Beverages"].loc["2000"],

                '20_t_tawa': props["downscaled_trade_value"]["Textiles and Wearing Apparel"].loc["2000"],
                '20_c_tawa': props["downscaled_impacts_from_production"]["Textiles and Wearing Apparel"].loc["2000"],

                '20_t_wap': props["downscaled_trade_value"]["Wood and Paper"].loc["2000"],
                '20_c_wap': props["downscaled_impacts_from_production"]["Wood and Paper"].loc["2000"],

                '20_t_pcnmp': props["downscaled_trade_value"]["Petroleum, Chemical and Non-Metallic Mineral Products"].loc["2000"],
                '20_c_pcnmp': props["downscaled_impacts_from_production"]["Petroleum, Chemical and Non-Metallic Mineral Products"].loc["2000"],

                '20_t_mp': props["downscaled_trade_value"]["Metal Products"].loc["2000"],
                '20_c_mp': props["downscaled_impacts_from_production"]["Metal Products"].loc["2000"],

                '20_t_eam': props["downscaled_trade_value"]["Electrical and Machinery"].loc["2000"],
                '20_c_eam': props["downscaled_impacts_from_production"]["Electrical and Machinery"].loc["2000"],

                '20_t_te': props["downscaled_trade_value"]["Transport Equipment"].loc["2000"],
                '20_c_te': props["downscaled_impacts_from_production"]["Transport Equipment"].loc["2000"],

                '20_t_om': props["downscaled_trade_value"]["Other Manufacturing"].loc["2000"],
                '20_c_om': props["downscaled_impacts_from_production"]["Other Manufacturing"].loc["2000"],

                '20_t_rcycl': props["downscaled_trade_value"]["Recycling"].loc["2000"],
                '20_c_rcycl': props["downscaled_impacts_from_production"]["Recycling"].loc["2000"],

                '20_t_egw': props["downscaled_trade_value"]["Electricity, Gas and Water"].loc["2000"],
                '20_c_egw': props["downscaled_impacts_from_production"]["Electricity, Gas and Water"].loc["2000"],

                '20_t_con': props["downscaled_trade_value"]["Construction"].loc["2000"],
                '20_c_con': props["downscaled_impacts_from_production"]["Construction"].loc["2000"],

                '20_t_mar': props["downscaled_trade_value"]["Maintenance and Repair"].loc["2000"],
                '20_c_mar': props["downscaled_impacts_from_production"]["Maintenance and Repair"].loc["2000"],

                '20_t_wt': props["downscaled_trade_value"]["Wholesale Trade"].loc["2000"],
                '20_c_wt': props["downscaled_impacts_from_production"]["Wholesale Trade"].loc["2000"],

                '20_t_rt': props["downscaled_trade_value"]["Retail Trade"].loc["2000"],
                '20_c_rt': props["downscaled_impacts_from_production"]["Retail Trade"].loc["2000"],

                # Careful, misspelled in EORA, restraurants
                '20_t_har': props["downscaled_trade_value"]["Hotels and Restraurants"].loc["2000"],
                '20_c_har': props["downscaled_impacts_from_production"]["Hotels and Restraurants"].loc["2000"],

                '20_t_xprt': props["downscaled_trade_value"]["Transport"].loc["2000"],
                '20_c_xprt': props["downscaled_impacts_from_production"]["Transport"].loc["2000"],

                '20_t_pat': props["downscaled_trade_value"]["Post and Telecommunications"].loc["2000"],
                '20_c_pat': props["downscaled_impacts_from_production"]["Post and Telecommunications"].loc["2000"],

                # Careful, misspelled in EORA, finacial
                '20_t_fiba': props["downscaled_trade_value"]["Finacial Intermediation and Business Activities"].loc["2000"],
                '20_c_fiba': props["downscaled_impacts_from_production"]["Finacial Intermediation and Business Activities"].loc["2000"],

                '20_t_pa': props["downscaled_trade_value"]["Public Administration"].loc["2000"],
                '20_c_pa': props["downscaled_impacts_from_production"]["Public Administration"].loc["2000"],

                '20_t_ehos': props["downscaled_trade_value"]["Education, Health and Other Services"].loc["2000"],
                '20_c_ehos': props["downscaled_impacts_from_production"]["Education, Health and Other Services"].loc["2000"],

                '20_t_ph': props["downscaled_trade_value"]["Private Households"].loc["2000"],
                '20_c_ph': props["downscaled_impacts_from_production"]["Private Households"].loc["2000"],

                '20_t_oth': props["downscaled_trade_value"]["Others"].loc["2000"],
                '20_c_oth': props["downscaled_impacts_from_production"]["Others"].loc["2000"],
                
                # There are no carbon emissions from re-export & re-import
                '20_t_reri': props["downscaled_trade_value"]["Re-export & Re-import"].loc["2000"],
                #'20_c_reri': props["downscaled_impacts_from_production"]["Re-export & Re-import"].loc["2000"],
                
                
                
                'pc_t_ag': props["per_change_downscaled_trade_value"]["Agriculture"],
                'pc_c_ag': props["per_change_downscaled_impacts_from_production"]["Agriculture"],

                'pc_t_fsh': props["per_change_downscaled_trade_value"]["Fishing"],
                'pc_c_fsh': props["per_change_downscaled_impacts_from_production"]["Fishing"],

                'pc_t_maq': props["per_change_downscaled_trade_value"]["Mining and Quarrying"],
                'pc_c_maq': props["per_change_downscaled_impacts_from_production"]["Mining and Quarrying"],

                'pc_t_fab': props["per_change_downscaled_trade_value"]["Food & Beverages"],
                'pc_c_fab': props["per_change_downscaled_impacts_from_production"]["Food & Beverages"],

                'pc_t_tawa': props["per_change_downscaled_trade_value"]["Textiles and Wearing Apparel"],
                'pc_c_tawa': props["per_change_downscaled_impacts_from_production"]["Textiles and Wearing Apparel"],

                'pc_t_wap': props["per_change_downscaled_trade_value"]["Wood and Paper"],
                'pc_c_wap': props["per_change_downscaled_impacts_from_production"]["Wood and Paper"],

                'pc_t_pcnmp': props["per_change_downscaled_trade_value"]["Petroleum, Chemical and Non-Metallic Mineral Products"],
                'pc_c_pcnmp': props["per_change_downscaled_impacts_from_production"]["Petroleum, Chemical and Non-Metallic Mineral Products"],

                'pc_t_mp': props["per_change_downscaled_trade_value"]["Metal Products"],
                'pc_c_mp': props["per_change_downscaled_impacts_from_production"]["Metal Products"],

                'pc_t_eam': props["per_change_downscaled_trade_value"]["Electrical and Machinery"],
                'pc_c_eam': props["per_change_downscaled_impacts_from_production"]["Electrical and Machinery"],

                'pc_t_te': props["per_change_downscaled_trade_value"]["Transport Equipment"],
                'pc_c_te': props["per_change_downscaled_impacts_from_production"]["Transport Equipment"],

                'pc_t_om': props["per_change_downscaled_trade_value"]["Other Manufacturing"],
                'pc_c_om': props["per_change_downscaled_impacts_from_production"]["Other Manufacturing"],

                'pc_t_rcycl': props["per_change_downscaled_trade_value"]["Recycling"],
                'pc_c_rcycl': props["per_change_downscaled_impacts_from_production"]["Recycling"],

                'pc_t_egw': props["per_change_downscaled_trade_value"]["Electricity, Gas and Water"],
                'pc_c_egw': props["per_change_downscaled_impacts_from_production"]["Electricity, Gas and Water"],

                'pc_t_con': props["per_change_downscaled_trade_value"]["Construction"],
                'pc_c_con': props["per_change_downscaled_impacts_from_production"]["Construction"],

                'pc_t_mar': props["per_change_downscaled_trade_value"]["Maintenance and Repair"],
                'pc_c_mar': props["per_change_downscaled_impacts_from_production"]["Maintenance and Repair"],

                'pc_t_wt': props["per_change_downscaled_trade_value"]["Wholesale Trade"],
                'pc_c_wt': props["per_change_downscaled_impacts_from_production"]["Wholesale Trade"],

                'pc_t_rt': props["per_change_downscaled_trade_value"]["Retail Trade"],
                'pc_c_rt': props["per_change_downscaled_impacts_from_production"]["Retail Trade"],

                # Careful, misspelled in EORA, restraurants
                'pc_t_har': props["per_change_downscaled_trade_value"]["Hotels and Restraurants"],
                'pc_c_har': props["per_change_downscaled_impacts_from_production"]["Hotels and Restraurants"],

                'pc_t_xprt': props["per_change_downscaled_trade_value"]["Transport"],
                'pc_c_xprt': props["per_change_downscaled_impacts_from_production"]["Transport"],

                'pc_t_pat': props["per_change_downscaled_trade_value"]["Post and Telecommunications"],
                'pc_c_pat': props["per_change_downscaled_impacts_from_production"]["Post and Telecommunications"],

                # Careful, misspelled in EORA, finacial
                'pc_t_fiba': props["per_change_downscaled_trade_value"]["Finacial Intermediation and Business Activities"],
                'pc_c_fiba': props["per_change_downscaled_impacts_from_production"]["Finacial Intermediation and Business Activities"],

                'pc_t_pa': props["per_change_downscaled_trade_value"]["Public Administration"],
                'pc_c_pa': props["per_change_downscaled_impacts_from_production"]["Public Administration"],

                'pc_t_ehos': props["per_change_downscaled_trade_value"]["Education, Health and Other Services"],
                'pc_c_ehos': props["per_change_downscaled_impacts_from_production"]["Education, Health and Other Services"],

                'pc_t_ph': props["per_change_downscaled_trade_value"]["Private Households"],
                'pc_c_ph': props["per_change_downscaled_impacts_from_production"]["Private Households"],

                'pc_t_oth': props["per_change_downscaled_trade_value"]["Others"],
                'pc_c_oth': props["per_change_downscaled_impacts_from_production"]["Others"],
                
                # There are no carbon emissions from re-export & re-import
                'pc_t_reri': props["per_change_downscaled_trade_value"]["Re-export & Re-import"],
                #'pc_c_reri': props["per_change_downscaled_impacts_from_production"]["Re-export & Re-import"]

                
                
                
            }
        })


